In [2]:
from pathlib import Path
import os
import pandas as pd

drive_path = Path("/run/user/1000/gvfs/google-drive:host=gmail.com,user=przemek.7678/Shared drives/SKN AI FUW /Projekt z Dagmarą Mateją/Korpusy/Korpus")

In [9]:
columns = ['id', 'title', 'source', 'author', 'department', 'date', 'text']

In [10]:
corp_dfs = []
for corp in os.listdir(drive_path):
    # Read the file
    try:
        df = pd.read_csv(drive_path / corp)
    except:
        print(f"Error reading {corp}")
        continue
    
    # Check if all columns are present
    for col in columns:
        if col not in df.columns:
            print(f"Column {col} not found in {corp}")
            if col == 'id':
                df['id'] = df.index
            else:
                df[col] = None

    # text is None
    df = df[df['text'].isna() == False]

    # Id is duplicated or missing
    if df.id.duplicated().sum() > 0 or df.id.isna().sum() > 0:
        print(f"Duplicated or missing index found in {corp}")
        df = df.reset_index()
        df['id'] = df.index
        
    # Source is not unique
    if len(df['source'].unique()) != 1:
        raise ValueError("Multiple sources in one file")
    
    # Select only the columns we want and fill NaN with None
    df = df[columns]
    df = df.fillna("None")

    corp_dfs.append(df)

Column department not found in 1k-Yp-YKK_m2MPGzArGTiVeDFKevQe8U3
Error reading 1DiYvyWfZfWU-3HdO5gC9KS_ImEP_fy0C
Column department not found in 1G8_M46Fkr1MTEqTlKQ1uIsPye9yDS1sr
Column id not found in 1mQy5b8qnSEhZh-Mkmz3o3H17nc3mqWo4
Column department not found in 1WhtapWRmozmnXtUGdtaAYHIy8Sy-YKve
Column id not found in 19QavqAYn6WeCoXba-DcolKEmuStbMo2N
Column department not found in 1fY6-IZIjWVuEWKVIctwurLOrQeWjGNGS


In [11]:
def to_sketch_format(df):
    for col in df.columns:
        df[col] = df[col].astype(str)
    sketch_text = "<doc text_id=\"" + df['id'] + "\" title=\"" + df['title'] + "\" source=\"" + df['source'] + "\" author=\"" + df['author'] + "\" department=\"" + df['department'] + "\" date=\"" + df['date'] + "\">" + df['text'] + "</doc>"
    return sketch_text


In [13]:
import re
corpus = 0
for df in corp_dfs:

    # Convert to sketch format
    df['sketch'] = to_sketch_format(df)
    
    # Save to file
    file = "sketch_" + df['source'][0] + ".txt"
    df['sketch'].to_csv(file, index=False, header=False)
    corpus += len(df)
    # Clean the file
    with open(file, "r") as f:
        text = f.read()
        text = re.sub(r'\"+', "\"", text)
        text = re.sub(r"\n", " ", text)
        text = re.sub(r"\s+", " ", text)
    with open(file, "w") as f:
        f.write(text)
        
print(corpus)

19429
